In [ ]:
using LinearAlgebra
using ForwardDiff 
using SparseArrays
using Plots

In [ ]:
#this is working

In [ ]:
#nx - size of state 
#n_eq - number of equality constraints 
#n_ineq - number of inequality constraints 
#fill ratio- (0,1], determines how sparse the QP is 

function gen_sparse_qp(nx, n_eq, n_ineq, fill_ratio)

    #QP of the form: 
    #0.5*x'*Q*x + q'*x 

    #transformed the typical equality constraint Gx <= h
    #into an equality by introducing the slack variable s = h - Gx
    #and we have a new inequality s >=0

    #equality constraint 
    # Ax-b = 0
    # Gx + s - h = 0

    #inequality constraint 
    # s >=0

    #generating the QP with the solution (an x that satisfies the KKT conditions)

    #μ is the dual for the equality constraints 
    #λ is the dual for the inequality constraints 

    #KKT conditions 

    #stationarity: 
    #Qx + q + A'*μ + G'*λ

    #primal feasibility
    # Ax - b = 0
    # Gx + s - h = 0 
    # s >= 0

    #dual feasibility 
    # λ >=0 

    # complimentarity condition 
    # λ_i*s_i = 0


    #create Q (quadratic cost scaling matrix)
    Q = sprandn(nx, nx, fill_ratio)

    #adding the identity ensures it is positive semidefnite
    Q = Q'*Q + I 

    #create A matrix (equality constraint)
    #Ax = b
    A = sprandn(n_eq, nx, fill_ratio)
    A[1:n_eq,1:n_eq] += I 

    #create G matrix (inequality constraint)
    G = sprandn(n_ineq, nx, fill_ratio)

    #primal variable 
    x = randn(nx)

    #dual variable on the equality constraints 
    μ = randn(n_eq)



    #logic from kevin 
    #######################################################################################
    #slack variable 
    s = zeros(n_ineq)

    #dual variable on the inequality constraints 
    λ = zeros(n_ineq)

    #if s is violating the constraint (s < 0), then the dual variable remains zero 
    #this logic is to ensure s and z satisfy the complimentarity condition 

    for i = 1:n_ineq
        if randn() < 0 
            s[i] = abs(randn())
        else
            λ[i] = abs(randn())
        end
    end
    ################################################################################################

    #logic from John
    #slack variable 
    # s = abs.(randn(n_ineq))

    # #dual variable on the inequality constraints 
    # λ = abs.(randn(n_ineq))


    # for i in 1:n_ineq
    #     if rand() > 0.5
    #         s[i] = 0
    #     else
    #         λ[i] = 0
    #     end 
        
    # end


    #create right hand size of equality constraint 
    b = A * x 

    #create right hand size of inequality constraint 
    h = G * x + s 

    #set q in such a way that it makes the stationarity condition equal to zero 
    q = -(Q*x + A'*μ + G'*λ)

    #check that all the KKT conditions are satisfied 

    #stationarity condition 
    @assert norm(Q*x + q + A'*μ + G'*λ) < 1e-10
    
    #complimentarity condition 
    @assert abs(s'*λ) < 1e-10
    
    #primal feasibility 
    @assert minimum(s) >= 0 
    @assert norm(A*x - b) < 1e-10 
    @assert norm(G*x + s - h) < 1e-10
    
    #dual feasibility 
    @assert minimum(λ) >= 0 

    println("created sparse QP successfully")

    Q, q, A, b, G, h, x, λ, μ, s

end

In [ ]:
#works for [3,3,2]
#works for [100, 100, 120]
#works for [1000, 1000, 1200]

#size of the state 
nx = 30

#number of equality constraints 
n_eq = 20

#number of inequality constraints 
n_ineq = 20

#this creates a dense QP since the ratio is 1.0
Q, q, A, b, G, h, x_solution, λ_solution, μ_solution, s_solution = gen_sparse_qp(nx, n_eq, n_ineq, 1.0)
Q = Matrix(Q)
A = Matrix(A)
G = Matrix(G)

#set up the problem parameters into a tuple
prob = (Q = Q, q = q, A = A, b=b, G = G, h=h, nx = nx, n_eq=n_eq, n_ineq=n_ineq)

In [ ]:
G*x_solution + s_solution - h

In [ ]:
s_solution

In [ ]:
λ_solution 

In [ ]:
#for the log domain, we eliminate the inequality constraints through a change of variables. 

In [ ]:
#residual function are the 3 KKT conditions that remain in the log domian 

#state z = [ x
#            μ
#            σ ]

#penalty parameter κ

function residual(prob, z, κ)

    #state 
    x = z[1:prob.nx] 

    #equality constraints dual 
    μ = z[prob.nx+1: prob.nx+prob.n_eq]

    #slack variable
    σ = z[prob.nx+prob.n_eq+1:end]


    res = [
           #stationarity condition 
           prob.Q*x + prob.q + prob.A'*μ + prob.G'*(sqrt(κ)*exp.(σ)); 

           #equality constraints
           prob.A*x - prob.b; 

           G*x + sqrt(κ)*exp.(-σ) - prob.h
           
          ]
    
    return res

end

In [ ]:
# z - state [x, μ, σ]
# κ - kappa 
#checked with forwardDiff
function residual_jacobian(prob, z, κ)

    #slack variable
    σ = z[prob.nx+prob.n_eq+1:end]
    
    res_jac = [
                
                Q       A'                              G'*diagm(sqrt(κ)*exp.(σ)); 
                A       zeros(prob.n_eq, prob.n_eq)     zeros(prob.n_eq, prob.n_ineq); 
                G       zeros(prob.n_ineq, prob.n_eq)   -diagm(sqrt(κ)*exp.(-σ))

              ]

    #println(size(res_jac), " ", rank(res_jac), " ", cond(res_jac))
    return res_jac

end

In [ ]:
function newton_step(prob, z, κ)

    Δz = -residual_jacobian(prob, z, κ)\residual(prob, z, κ)

    return Δz

end

In [ ]:
function newtons_method_solve(prob, z, κ, max_iters)

    #define parameters for linesearch 
    b = 1e-1
    c = 0.5


    #initialize the step direction to 1
    α = 1

    #matrix to save all the states per iteration 
    z_all = zeros(size(z)[1], max_iters)

    all_residuals = zeros(max_iters)

    #set initial state 
    z_all[:,1] = z

    #set initial residual
    all_residuals[1] = norm(residual(prob, z_all[:,1], κ))

    #set the iteration counter
    iters = 1

    for i = 1:max_iters-1

        #this was the fix
        α = 1

        iters += 1

        #calculate the step 
        Δz = newton_step(prob, z_all[:,i], κ)
        # Δz = - residual_jacobian(prob, z_all[:,i], κ) \ residual(prob, z_all[:,i], κ)
        #println(norm(residual_jacobian(prob, z_all[:,i], κ)*-Δz - residual(prob, z_all[:,i], κ)))
        #println(sort(abs.(eigvals(residual_jacobian(prob, z_all[:,i], κ)))))

        #apply a linesearch for up to 10 iterations 
        for k = 1:10

            #armijo linesearch
            #compares the difference in residual of the new step to the reduction using a linear approximation of the function 
            # if norm(residual(prob, z_all[:,i] + α*Δz, κ)) >  norm(residual(prob, z_all[:,i], κ)) #+ b*α*residual_jacobian(prob, z_all[:,i], κ)'*Δz)
            if norm(residual(prob, z_all[:,i] + α*Δz, κ)) >  norm(residual(prob, z_all[:,i], κ) + b*α*residual_jacobian(prob, z_all[:,i], κ)'*Δz)
                
                println("in here")
                #reduce the step size
                α = c*α

            else
                
                break

            end

        end


        #update the state
        z_all[:,i+1] = z_all[:,i] + α*Δz 

        #check the norm of the residual on the updated state 
        all_residuals[i+1] = norm(residual(prob, z_all[:,i+1], κ))

        println("this is all residuals: ", all_residuals[i+1])
        
        #if we achieve the tolerance, then break the loop 
        if all_residuals[i+1] < newton_tol 

            println("tolerance achieved")
            break

        end

    end

    #truncate to just the iteration count 
    z_all = z_all[:, 1:iters]

    all_residuals = all_residuals[1:iters]

    return z_all, all_residuals
end

In [ ]:
#set the tolerance 
#works with 1e-6
newton_tol = 1e-8

In [ ]:
#solve test 
#newton seems to be working
# z0 = randn(nx + n_eq + n_ineq)
# κ0 = 1
# max_iters = 20


# z_all, all_residuals = newtons_method_solve(prob, z0, κ0, max_iters)

In [ ]:
#initial guess
z0 = randn(nx + n_eq + n_ineq)
κ0 = 1
max_iters = 20

#works for 3
solve_iters = 10

#penalty will be divided by this after every iteration
reduction = 10

initial_guesses = zeros(nx + n_eq + n_ineq, solve_iters)

penalty_parameters = zeros(solve_iters)

penalty_parameters[1] = κ0

#initial guess for the first solve 
initial_guesses[:,1] = z0

#there is a newton solve for every penalty (central path) parameter κ
for i=1:solve_iters-1

    z_all, all_residuals = newtons_method_solve(prob, initial_guesses[:,i], penalty_parameters[i], max_iters)

    #the initial guess for the next iteration is the solution of the previous iteration 
    initial_guesses[:,i+1] = z_all[:,end] 

    #penalty for the next term is reduced 
    penalty_parameters[i+1] = penalty_parameters[i]/reduction

end


In [ ]:
initial_guesses 

In [ ]:
penalty_parameters 

In [ ]:
#the solution is the final solve (initial guess update)
solver_solution = initial_guesses[:,end]

In [ ]:
#the residual of the solution is extremely small
plot(residual(prob, solver_solution, penalty_parameters[end-1])) 

In [ ]:
#this is the true solution. the one generated with the QP 
x_solution

In [ ]:
#lambdas from the solver 
λ_solver = sqrt(penalty_parameters[end-1])*exp.(solver_solution[nx+n_eq+1:end])

In [ ]:
#k=0 is not feasible. always evaluate at the final penalty 

In [ ]:
#compare solution of state to the true solution
#tolerance to 1e-6

#struggles for tighter tolerances 
primal_solution_accuracy = norm(solver_solution[1:nx] - x_solution)

In [ ]:
dual_equality_solution_accuracy = norm(solver_solution[prob.nx+1: prob.nx+prob.n_eq] - μ_solution ) 

In [ ]:
dual_inequality_solution_accuracy = norm(λ_solver - λ_solution) 

In [ ]:
#this is the initial residual
norm(residual(prob, z0, κ0))  

In [ ]:
#this is the final residual
norm(residual(prob, solver_solution, penalty_parameters[end-1]))  

In [ ]:
#residual doesn't go down to tight tolerances, stays in the 100's magnitude but the alg still finds the correct state. 
#maybe it is the scaling of the problem 

#how do I get the solution, if the kkt optimality conditions aren't being solved to tight tolerances? 